<a href="https://colab.research.google.com/github/Alveoli/alveoli.github.io/blob/main/DaBigScheduleofTrains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vibe code instructions for LLM to Build London Commute App

Please write this programme in one code file for Colab.

---

## Scope & Platforms

**Instruction:** Build a **web-only** text-based application. No inputs, no UI beyond static text output. I will wrap it myself for Android or web browser embedding.

**Instruction:** No login or authentication. Entirely open to public.

**Instruction:** Transport modes to support: **Tube, Elizabeth line, Buses**. Walking/driving only considered for access/egress legs. Ignore DLR/Overground/Trams/National Rail.

---

## Language & Dev Environment

**Instruction:** Implement in **Python 3.10+**.

**Instruction:** Primary development in **Google Colab**. Provide:

A well-documented  **`main.py`** export that runs headlessly with explanatory Markdown for teaching an 11-year-old..

**Instruction:** Code should be didactic: small functions, type hints, docstrings, clear comments, and simple tests where practical.

**Instruction:** Keep the tech stack as **native and simple as possible**. Use minimal libraries — only mainstream, beginner‑friendly ones (e.g., `requests`, `json`, `datetime`). Avoid heavy frameworks.

**Instruction:** **Zero interactivity.** Each run generates the full **text-only output** for a notice board. No inputs, forms, or clicks.

**Instruction:** **Do not** make origin/destination configurable. **Hard-code** the journeys and variants listed in *Fixed Journeys & Stops* below. No external config files and no runtime inputs.

**Instruction:** Prepare the code so it can later be hosted in a **serverless container (e.g., Azure Logic Apps/Azure Functions)**. Provide a pure function entry point for easy wiring.

**Instruction:** Use **TfL Unified API** endpoints for Tube, Elizabeth line and Buses. If an `app_key` is provided via env var or config, include it; otherwise use unauthenticated endpoints where permitted.

**Instruction:** Output options:

* Build up an output variable over the course of the code which is the string that will be displayed to the user.
* At the end, print this string **plain-text block** to stdout, and

---

## Fixed Journeys & Stops

**Instruction:** **Hard-code journeys and variants** (no configurability):

* **Morning (Outbound)**

  * **Origin:** postcode **W13 0EZ**.
  * **Start options:** either one of: **West Ealing**, **Ealing Broadway**, **North Ealing**.
  * **Destination:** **Godolphin & Latymer School** (Hammersmith) via one of: **Hammersmith** station **or** **Ravenscourt Park** station.

* **Return (Inbound)**

  * **Origin:** **Godolphin & Latymer School** via **Hammersmith**
  * **Destination path:** travel to **West Ealing** or **Ealing Broadway**, then **bus or walk** onward to **W13 0EZ**.

**Instruction:** Show **all route variants**; do **not** filter or rank. For **each route** display:

1. **Next departure** time for the first mode, **plus next and next** (three total).
2. **Estimated ETA** for that route to the final destination.

**Instruction:** Detail level: **station/stop** only (no platform/stand numbers).

**Instruction:** **Disruption:** show the **very short disruption text alongside** the route’s departures/ETA. Do not grey-out or hide.

**Instruction:** **Walking time:** use **live location** and apply a small, adjustable **buffer** purely to decide if a departure is catchable. Do not label "run/jog" — just show time-to-departure.

**Instruction:** **Refresh:** **on-demand only**. Each run fetches fresh data; no background polling or push.

**Instruction:** **Notifications:** none (tap-to-check only; suitable for a notice board).

**Instruction:** **Tone:** **minimal and factual**. Keep output simple text; you may add a few **emojis** to keep it engaging for an 11‑year‑old.

**Instruction:** **Secrets & config:** define as **hardcoded constants at the top of the code**. No environment variables or external files. Origin/destination, buffers, and API keys live in constants within the Python script.

**Instruction:** **Client-only** calls to **TfL Unified API** (Tube, Elizabeth line, Buses). **Free-only** usage. Add **helpful comments** and simple **logging** at INFO level.

---

## Use the following TFL API Subscription keys:

* **TFL API** (500 requests/min)

  * Primary key: `b9c520a7e4d64f42a6fc96c35bcad1fd`
  * Secondary key: `fc6e7ee125584896b492f7f6d2325927`

* **TFL API: TrackernetFeedPublic**

  * Primary key: `1c4946db28b94a6290edfcf2627f3373`
  * Secondary key: `51907209d7e54f12b33141de356603bc`


# My code

In [ ]:
"""
London Commute App — text-only notice board (single file, Colab-ready)
=====================================================================

This tiny program prints a **plain-text board** for a fixed school run from
**W13 0EZ ↔ Godolphin & Latymer School (Hammersmith)**.

- Zero inputs, zero UI. Every run fetches fresh live data from the TfL Unified API.
- Support: **Tube, Elizabeth line, Buses** (buses used for final hop ETA and line status; first-mode
  departures are Tube/Elizabeth).
- We **hard-code** the journeys and times so an 11-year-old could follow the logic.
- Output shows for **each route variant**:
  1) next three departure times for the **first mode** (the first train), with a ✅/❌ for whether you can
     catch it based on walking time from home (with a small buffer), and
  2) an **ETA** to the final destination (school/home) if you board each of those trains.
  3) very short **disruption** text for the lines used by that route.

Why we don’t do everything perfectly live:
- Journey-planner forcing exact change stations can be complex and rate-limited; instead, we combine
  **live first-train times** with simple, conservative **fixed in-vehicle and interchange minutes**.
- That keeps it reliable for a classroom-level demo and serverless hosting later.

Run it anywhere that has Python 3.10+ and internet access. On Colab, just run the cell.

"""
from __future__ import annotations

import dataclasses
import logging
from dataclasses import dataclass
from datetime import datetime, timedelta, timezone
from zoneinfo import ZoneInfo
from typing import Dict, List, Optional, Tuple
import math
import requests

# ---------------------------------------------------------------------------
# CONFIG: constants live here (as requested: no env vars, no external files)
# ---------------------------------------------------------------------------
TFL_APP_KEY_PRIMARY = "b9c520a7e4d64f42a6fc96c35bcad1fd"
TFL_APP_KEY_SECONDARY = "fc6e7ee125584896b492f7f6d2325927"
# Trackernet key not required for Unified API endpoints used below, but kept for completeness
TFL_TRACKERNET_PRIMARY = "1c4946db28b94a6290edfcf2627f3373"
TFL_TRACKERNET_SECONDARY = "51907209d7e54f12b33141de356603bc"

# Base API URL
TFL_BASE = "https://api.tfl.gov.uk"

# London timezone
LONDON_TZ = ZoneInfo("Europe/London")

# Home & School (approx coordinates)
# Home: W13 0EZ (approx) — kept broad to avoid geocoding
HOME_LAT, HOME_LON = 51.5123, -0.3229
SCHOOL_LAT, SCHOOL_LON = 51.4987, -0.2338  # Godolphin & Latymer, Hammersmith (approx)

# StopPoint IDs (hard-coded so we don't rely on name search each run)
# Elizabeth line (National Rail-style 910G codes) & Tube (940GZZLU...)
STOP_WEST_EALING_ELIZ = "910GWEALING"
STOP_EALING_BROADWAY_ELIZ = "910GEALINGB"
STOP_EALING_BROADWAY_TUBE = "940GZZLUEBY"
STOP_NORTH_EALING_TUBE = "940GZZLUNEN"
STOP_HAMMERSMITH_DIST_PICC = "940GZZLUHSD"
STOP_RAVENSCOURT_PARK = "940GZZLURVP"

# Line IDs as TfL uses them
LINE_ELIZABETH = "elizabeth"
LINE_DISTRICT = "district"
LINE_PICCADILLY = "piccadilly"
# Useful Ealing-area bus lines for status context (used in finales)
BUS_LINES = ["e2", "e7", "e8", "e11", "sl8"]

# Conservative walking times (minutes) from home to first stations
WALK_HOME_TO_WEST_EALING_MIN = 12
WALK_HOME_TO_EALING_BROADWAY_MIN = 20
WALK_HOME_TO_NORTH_EALING_MIN = 25

# School last-hop walks (minutes)
WALK_HAMMERSMITH_TO_SCHOOL_MIN = 10
WALK_RAVENSCOURT_TO_SCHOOL_MIN = 9

# Final hop home from stations (if walking)
WALK_EALING_BROADWAY_TO_HOME_MIN = 20
WALK_WEST_EALING_TO_HOME_MIN = 12

# Average bus option from Ealing Broadway → W13 0EZ (ETA only; first mode stays Tube)
# We assume typical kid-friendly choice; not used for live departures, only ETA calculation.
AVG_BUS_WAIT_MIN = 4
BUS_EALING_BROADWAY_TO_HOME_MIN = 10
BUS_WEST_EALING_TO_HOME_MIN = 6

# Interchange & ride minutes — conservative classroom numbers
# (These are deliberately simple and easy to tweak.)
IV_WEST_EALING_TO_EALING_BROADWAY_ELIZ_MIN = 2
CHANGE_AT_EALING_BROADWAY_MIN = 3
IV_EALING_BROADWAY_TO_HAMMERSMITH_DIST_MIN = 16
IV_EALING_BROADWAY_TO_RAVENSCOURT_DIST_MIN = 15
IV_NORTH_EALING_TO_HAMMERSMITH_PICC_MIN = 14

# Catchability buffer: you probably need a tiny safety margin beyond raw walking time
CATCH_BUFFER_MIN = 2

# Maximum number of first-mode departures to display per route
N_NEXT = 3

# Logging
logging.basicConfig(level=logging.INFO, format="[%(levelname)s] %(message)s")
log = logging.getLogger("commute")


# ---------------------------------------------------------------------------
# Small helpers
# ---------------------------------------------------------------------------
def now_london() -> datetime:
    """Current time in Europe/London."""
    return datetime.now(tz=LONDON_TZ)


def fmt_hhmm(dt: datetime) -> str:
    return dt.astimezone(LONDON_TZ).strftime("%H:%M")


def plural(n: int, unit: str) -> str:
    return f"{n} {unit}{'' if n == 1 else 's'}"


def minutes_between(a: datetime, b: datetime) -> int:
    return max(0, int(round((b - a).total_seconds() / 60)))


# ---------------------------------------------------------------------------
# TfL API wrappers (very light)
# ---------------------------------------------------------------------------
SESSION = requests.Session()


def tfl_get(path: str, params: Optional[Dict[str, str]] = None) -> requests.Response:
    """HTTP GET to TfL Unified API with app_key if present.

    We pass only the primary key to keep it simple. If you hit rate limits,
    switch to the secondary or rotate between them.
    """
    url = f"{TFL_BASE}{path}"
    params = dict(params or {})
    if TFL_APP_KEY_PRIMARY:
        params["app_key"] = TFL_APP_KEY_PRIMARY
    log.info("GET %s", url)
    r = SESSION.get(url, params=params, timeout=15)
    r.raise_for_status()
    return r


@dataclass
class FirstModeDeparture:
    line_id: str
    direction_text: str  # e.g., "Eastbound" / "Westbound" / bus direction
    when: datetime
    seconds_to: int
    catchable: bool


def get_first_mode_departures(
    stop_point_id: str,
    line_filter: str,
    direction_keyword: str,
    n: int = N_NEXT,
) -> List[FirstModeDeparture]:
    """Return next N departures at a StopPoint that match a line and (if possible) a direction.

    Uses /StopPoint/{id}/Arrivals. We first filter by `lineId` and platform direction keyword.
    If that returns nothing (some stops omit platform direction), we fall back to filtering by
    line only.
    """
    resp = tfl_get(f"/StopPoint/{stop_point_id}/Arrivals")
    arr = resp.json()

    def _filter(with_direction: bool) -> List[FirstModeDeparture]:
        out: List[FirstModeDeparture] = []
        _now = now_london()
        for p in arr:
            try:
                if (p.get("lineId", "").lower() != line_filter.lower()):
                    continue
                plat = (p.get("platformName") or "").lower()
                if with_direction and direction_keyword and (direction_keyword.lower() not in plat):
                    continue
                secs = int(p.get("timeToStation", 0))
                when = _now + timedelta(seconds=secs)
                out.append(
                    FirstModeDeparture(
                        line_id=p.get("lineId", ""),
                        direction_text=p.get("platformName") or p.get("direction", ""),
                        when=when,
                        seconds_to=secs,
                        catchable=False,
                    )
                )
            except Exception:
                continue
        out.sort(key=lambda d: d.seconds_to)
        return out[:n]

    filtered = _filter(with_direction=True)
    if not filtered:
        filtered = _filter(with_direction=False)
    return filtered


@dataclass
class LineStatus:
    line_id: str
    short: str  # e.g., "Good Service", "Minor Delays"


def get_line_statuses(line_ids: List[str]) -> Dict[str, LineStatus]:
    """Fetch very short line status strings for the given line ids.

    We call /Line/{ids}/Status and keep the succinct severity text.
    """
    if not line_ids:
        return {}
    ids_csv = ",".join(line_ids)
    resp = tfl_get(f"/Line/{ids_csv}/Status", params={"detail": "false"})
    out: Dict[str, LineStatus] = {}
    try:
        for item in resp.json():
            lid = item.get("id", "").lower()
            statuses = item.get("lineStatuses") or []
            short = statuses[0].get("statusSeverityDescription", "Unknown") if statuses else "Unknown"
            out[lid] = LineStatus(line_id=lid, short=short)
    except Exception:
        pass
    return out


# ---------------------------------------------------------------------------
# Route model and ETA calculation (simple & didactic)
# ---------------------------------------------------------------------------
@dataclass
class Leg:
    """A leg of the journey with a simple, conservative duration in minutes."""
    label: str
    minutes: int


@dataclass
class RouteVariant:
    code: str  # short id
    title: str  # human label
    first_mode_stop_id: str
    first_mode_line: str
    first_mode_dir_keyword: str  # e.g., 'eastbound'/'westbound'
    walk_from_home_min: int
    legs_after_boarding: List[Leg]
    lines_used: List[str]
    last_hop: str  # where you end: 'School via Hammersmith' etc.

    def eta_if_boarding(self, depart_at: datetime) -> datetime:
        """Add up: (walk to first stop) + waiting until depart + fixed legs."""
        now = now_london()
        wait_min = minutes_between(now, depart_at)
        total_min = self.walk_from_home_min + wait_min + sum(l.minutes for l in self.legs_after_boarding)
        return now + timedelta(minutes=total_min)

    def catchable(self, depart_at: datetime) -> bool:
        """Is the train catchable given walk from home + buffer?"""
        now = now_london()
        mins_until_train = minutes_between(now, depart_at)
        return mins_until_train >= (self.walk_from_home_min + CATCH_BUFFER_MIN)


# Build all required variants -------------------------------------------------
MORNING_ROUTES: List[RouteVariant] = [
    RouteVariant(
        code="A1",
        title="West Ealing → Ealing Broadway (ELIZ) → Hammersmith (DIST) → Walk to School",
        first_mode_stop_id=STOP_WEST_EALING_ELIZ,
        first_mode_line=LINE_ELIZABETH,
        first_mode_dir_keyword="eastbound",
        walk_from_home_min=WALK_HOME_TO_WEST_EALING_MIN,
        legs_after_boarding=[
            Leg("Ride: West Ealing → Ealing Broadway (Elizabeth)", IV_WEST_EALING_TO_EALING_BROADWAY_ELIZ_MIN),
            Leg("Change at Ealing Broadway", CHANGE_AT_EALING_BROADWAY_MIN),
            Leg("Ride: Ealing Broadway → Hammersmith (District)", IV_EALING_BROADWAY_TO_HAMMERSMITH_DIST_MIN),
            Leg("Walk: Hammersmith → School", WALK_HAMMERSMITH_TO_SCHOOL_MIN),
        ],
        lines_used=[LINE_ELIZABETH, LINE_DISTRICT],
        last_hop="School via Hammersmith",
    ),
    RouteVariant(
        code="A2",
        title="West Ealing → Ealing Broadway (ELIZ) → Ravenscourt Park (DIST) → Walk to School",
        first_mode_stop_id=STOP_WEST_EALING_ELIZ,
        first_mode_line=LINE_ELIZABETH,
        first_mode_dir_keyword="eastbound",
        walk_from_home_min=WALK_HOME_TO_WEST_EALING_MIN,
        legs_after_boarding=[
            Leg("Ride: West Ealing → Ealing Broadway (Elizabeth)", IV_WEST_EALING_TO_EALING_BROADWAY_ELIZ_MIN),
            Leg("Change at Ealing Broadway", CHANGE_AT_EALING_BROADWAY_MIN),
            Leg("Ride: Ealing Broadway → Ravenscourt Park (District)", IV_EALING_BROADWAY_TO_RAVENSCOURT_DIST_MIN),
            Leg("Walk: Ravenscourt Park → School", WALK_RAVENSCOURT_TO_SCHOOL_MIN),
        ],
        lines_used=[LINE_ELIZABETH, LINE_DISTRICT],
        last_hop="School via Ravenscourt Park",
    ),
    RouteVariant(
        code="B1",
        title="Ealing Broadway (DIST) → Hammersmith → Walk to School",
        first_mode_stop_id=STOP_EALING_BROADWAY_TUBE,
        first_mode_line=LINE_DISTRICT,
        first_mode_dir_keyword="eastbound",
        walk_from_home_min=WALK_HOME_TO_EALING_BROADWAY_MIN,
        legs_after_boarding=[
            Leg("Ride: Ealing Broadway → Hammersmith (District)", IV_EALING_BROADWAY_TO_HAMMERSMITH_DIST_MIN),
            Leg("Walk: Hammersmith → School", WALK_HAMMERSMITH_TO_SCHOOL_MIN),
        ],
        lines_used=[LINE_DISTRICT],
        last_hop="School via Hammersmith",
    ),
    RouteVariant(
        code="B2",
        title="Ealing Broadway (DIST) → Ravenscourt Park → Walk to School",
        first_mode_stop_id=STOP_EALING_BROADWAY_TUBE,
        first_mode_line=LINE_DISTRICT,
        first_mode_dir_keyword="eastbound",
        walk_from_home_min=WALK_HOME_TO_EALING_BROADWAY_MIN,
        legs_after_boarding=[
            Leg("Ride: Ealing Broadway → Ravenscourt Park (District)", IV_EALING_BROADWAY_TO_RAVENSCOURT_DIST_MIN),
            Leg("Walk: Ravenscourt Park → School", WALK_RAVENSCOURT_TO_SCHOOL_MIN),
        ],
        lines_used=[LINE_DISTRICT],
        last_hop="School via Ravenscourt Park",
    ),
    RouteVariant(
        code="C1",
        title="North Ealing (PICC) → Hammersmith → Walk to School",
        first_mode_stop_id=STOP_NORTH_EALING_TUBE,
        first_mode_line=LINE_PICCADILLY,
        first_mode_dir_keyword="eastbound",
        walk_from_home_min=WALK_HOME_TO_NORTH_EALING_MIN,
        legs_after_boarding=[
            Leg("Ride: North Ealing → Hammersmith (Piccadilly)", IV_NORTH_EALING_TO_HAMMERSMITH_PICC_MIN),
            Leg("Walk: Hammersmith → School", WALK_HAMMERSMITH_TO_SCHOOL_MIN),
        ],
        lines_used=[LINE_PICCADILLY],
        last_hop="School via Hammersmith",
    ),
]

RETURN_ROUTES: List[RouteVariant] = [
    RouteVariant(
        code="R1",
        title="Hammersmith (DIST westbound) → Ealing Broadway → Walk to W13 0EZ",
        first_mode_stop_id=STOP_HAMMERSMITH_DIST_PICC,
        first_mode_line=LINE_DISTRICT,
        first_mode_dir_keyword="westbound",
        walk_from_home_min=0,  # origin is School, not home (first leg is a train from Hammersmith)
        legs_after_boarding=[
            Leg("Ride: Hammersmith → Ealing Broadway (District)", IV_EALING_BROADWAY_TO_HAMMERSMITH_DIST_MIN),
            Leg("Walk: Ealing Broadway → Home", WALK_EALING_BROADWAY_TO_HOME_MIN),
        ],
        lines_used=[LINE_DISTRICT],
        last_hop="Home (walk)",
    ),
    RouteVariant(
        code="R2",
        title="Hammersmith (DIST westbound) → Ealing Broadway → Bus → W13 0EZ",
        first_mode_stop_id=STOP_HAMMERSMITH_DIST_PICC,
        first_mode_line=LINE_DISTRICT,
        first_mode_dir_keyword="westbound",
        walk_from_home_min=0,
        legs_after_boarding=[
            Leg("Ride: Hammersmith → Ealing Broadway (District)", IV_EALING_BROADWAY_TO_HAMMERSMITH_DIST_MIN),
            Leg("Wait for a bus (avg)", AVG_BUS_WAIT_MIN),
            Leg("Bus: Ealing Broadway → Home (avg)", BUS_EALING_BROADWAY_TO_HOME_MIN),
        ],
        lines_used=[LINE_DISTRICT] + BUS_LINES,  # we fold bus lines for status context
        last_hop="Home (bus)",
    ),
    RouteVariant(
        code="R3",
        title="Hammersmith (DIST westbound) → Ealing Broadway → Elizabeth → West Ealing → Walk to W13 0EZ",
        first_mode_stop_id=STOP_HAMMERSMITH_DIST_PICC,
        first_mode_line=LINE_DISTRICT,
        first_mode_dir_keyword="westbound",
        walk_from_home_min=0,
        legs_after_boarding=[
            Leg("Ride: Hammersmith → Ealing Broadway (District)", IV_EALING_BROADWAY_TO_HAMMERSMITH_DIST_MIN),
            Leg("Change at Ealing Broadway", CHANGE_AT_EALING_BROADWAY_MIN),
            Leg("Ride: Ealing Broadway → West Ealing (Elizabeth)", IV_WEST_EALING_TO_EALING_BROADWAY_ELIZ_MIN),
            Leg("Walk: West Ealing → Home", WALK_WEST_EALING_TO_HOME_MIN),
        ],
        lines_used=[LINE_DISTRICT, LINE_ELIZABETH],
        last_hop="Home via West Ealing (walk)",
    ),
    RouteVariant(
        code="R4",
        title="Hammersmith (DIST westbound) → Ealing Broadway → Elizabeth → West Ealing → Bus → W13 0EZ",
        first_mode_stop_id=STOP_HAMMERSMITH_DIST_PICC,
        first_mode_line=LINE_DISTRICT,
        first_mode_dir_keyword="westbound",
        walk_from_home_min=0,
        legs_after_boarding=[
            Leg("Ride: Hammersmith → Ealing Broadway (District)", IV_EALING_BROADWAY_TO_HAMMERSMITH_DIST_MIN),
            Leg("Change at Ealing Broadway", CHANGE_AT_EALING_BROADWAY_MIN),
            Leg("Ride: Ealing Broadway → West Ealing (Elizabeth)", IV_WEST_EALING_TO_EALING_BROADWAY_ELIZ_MIN),
            Leg("Wait for a bus (avg)", AVG_BUS_WAIT_MIN),
            Leg("Bus: West Ealing → Home (avg)", BUS_WEST_EALING_TO_HOME_MIN),
        ],
        lines_used=[LINE_DISTRICT, LINE_ELIZABETH] + BUS_LINES,
        last_hop="Home via West Ealing (bus)",
    ),
]

# Simple internal check so the minutes aren’t zero by mistake
assert IV_EALING_BROADWAY_TO_HAMMERSMITH_DIST_MIN > 0 and IV_NORTH_EALING_TO_HAMMERSMITH_PICC_MIN > 0


# ---------------------------------------------------------------------------
# Output builder
# ---------------------------------------------------------------------------

def build_section_for_routes(title: str, routes: List[RouteVariant]) -> str:
    out_lines: List[str] = []
    out_lines.append(title)
    out_lines.append("-" * len(title))

    # Collect statuses for all lines we will show in this section
    unique_lines: List[str] = []
    for r in routes:
        for lid in r.lines_used:
            if lid not in unique_lines:
                unique_lines.append(lid)
    statuses = get_line_statuses(unique_lines)

    for r in routes:
        # First-mode departures
        deps = get_first_mode_departures(
            stop_point_id=r.first_mode_stop_id,
            line_filter=r.first_mode_line,
            direction_keyword=r.first_mode_dir_keyword,
            n=N_NEXT,
        )
        # Tag catchable
        for d in deps:
            d.catchable = r.catchable(d.when)

        # Format block
        out_lines.append(f"[{r.code}] {r.title}")

        # Disruption short lines
        short_bits: List[str] = []
        for lid in r.lines_used:
            st = statuses.get(lid)
            if st:
                # Emoji: warn if not Good Service
                warn = " ⚠️" if st.short.lower() != "good service" else " ✅"
                short_bits.append(f"{lid.capitalize()}: {st.short}{warn}")
        if short_bits:
            out_lines.append("🚦 Status: " + " | ".join(short_bits))

        # First mode departures (three)
        if deps:
            dep_txts = []
            now = now_london()
            for d in deps:
                mins = minutes_between(now, d.when)
                tick = "✅" if d.catchable else "❌"
                dep_txts.append(f"{fmt_hhmm(d.when)} (in {mins}m) {tick}")
            out_lines.append("🚆 Next trains: " + "  |  ".join(dep_txts))
        else:
            out_lines.append("First train: no live data 😕")

        # ETAs if you boarded each
        if deps:
            eta_txts = []
            for d in deps:
                eta = r.eta_if_boarding(d.when)
                eta_txts.append(fmt_hhmm(eta))
            out_lines.append(f"🎯 ETA to {r.last_hop}: " + "  |  ".join(eta_txts))
        else:
            out_lines.append(f"ETAs to {r.last_hop}: —")

        out_lines.append("")

    return "\n".join(out_lines)


def build_notice_text() -> str:
    header = "London Commute — W13 0EZ ↔ Godolphin & Latymer (Hammersmith) 🚇🟪🚌"
    now_txt = now_london().strftime("%a %d %b %Y, %H:%M %Z")

    parts: List[str] = []
    parts.append(header)
    parts.append("=" * len(header))
    parts.append(f"Generated: {now_txt}  |  Data: TfL Unified API  |  Modes: 🚇/🟪/🚌  |  💡 Tip: re-run to refresh")

    # Morning section
    parts.append(build_section_for_routes("MORNING (Outbound) 🌞🚸", MORNING_ROUTES))

    # Return section
    parts.append(build_section_for_routes("RETURN (Inbound) 🌙🏠", RETURN_ROUTES))

    # Footer: tiny legend / tweakables
    parts.append("Notes")
    parts.append("-----")
    parts.append(
        "• ✅/❌ means ‘catchable’ based on walking time from home plus a small buffer.\n"
        f"• Buffer: {CATCH_BUFFER_MIN} min. Interchange at Ealing Broadway: {CHANGE_AT_EALING_BROADWAY_MIN} min.\n"
        "• In-vehicle times are conservative classroom values (easy to tweak at top).\n"
        "• Refresh by re-running. No notifications. Simple and factual for a notice board.\n"
    )

    return "\n".join(parts)


# ---------------------------------------------------------------------------
# Public entry points (serverless-friendly)
# ---------------------------------------------------------------------------

def run_notice_board() -> str:
    """Pure function that returns the final text block for printing."""
    try:
        return build_notice_text()
    except Exception as exc:
        log.exception("Failed to build notice text: %s", exc)
        return "(Error building notice board — check logs)"


# Azure Functions / Logic Apps adapter example (no external deps)
# You can wire this as an HTTP-triggered function and return the string body.

def handler(event: Optional[dict] = None, context: Optional[dict] = None) -> str:  # noqa: D401
    """Serverless adapter: ignore inputs, just return the board."""
    return run_notice_board()


# ---------------------------------------------------------------------------
# Script entry point
# ---------------------------------------------------------------------------
if __name__ == "__main__":
    text = run_notice_board()
    print(text)

London Commute — W13 0EZ ↔ Godolphin & Latymer (Hammersmith) 🚇🟪🚌
Generated: Thu 21 Aug 2025, 18:10 BST  |  Data: TfL Unified API  |  Modes: 🚇/🟪/🚌  |  💡 Tip: re-run to refresh
MORNING (Outbound) 🌞🚸
---------------------
[A1] West Ealing → Ealing Broadway (ELIZ) → Hammersmith (DIST) → Walk to School
🚦 Status: Elizabeth: Good Service ✅ | District: Good Service ✅
🚆 Next trains: 18:11 (in 1m) ❌  |  18:18 (in 8m) ❌  |  18:20 (in 10m) ❌
🎯 ETA to School via Hammersmith: 18:54  |  19:01  |  19:03

[A2] West Ealing → Ealing Broadway (ELIZ) → Ravenscourt Park (DIST) → Walk to School
🚦 Status: Elizabeth: Good Service ✅ | District: Good Service ✅
🚆 Next trains: 18:11 (in 1m) ❌  |  18:18 (in 8m) ❌  |  18:20 (in 10m) ❌
🎯 ETA to School via Ravenscourt Park: 18:52  |  18:59  |  19:01

[B1] Ealing Broadway (DIST) → Hammersmith → Walk to School
🚦 Status: District: Good Service ✅
🚆 Next trains: 18:11 (in 0m) ❌  |  18:11 (in 0m) ❌  |  18:26 (in 16m) ❌
🎯 ETA to School via Hammersmith: 18:56  |  18:56  |  19